## Name: Ishaun Sinha
## IPS Remote Internship
### Project: Fake News Detection 
### Algorithm used: Logistic Regression, Tfid Vectorizer

### Importing the libraries

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix



In [41]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [42]:
print(stopwords.words('English'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

### Reading the given data

In [43]:
df= pd.read_csv("news.csv")

### First 5 data

In [44]:
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


### Shape of the dataset

In [45]:
df.shape

(6335, 4)

In [46]:
from sklearn import preprocessing
label_encoder= preprocessing.LabelEncoder()
df['label']= label_encoder.fit_transform(df['label'])
df['label'].unique()

array([0, 1])

In [47]:
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",0
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,0
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,1
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",0
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,1


### Checking for null value

In [48]:
df.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [49]:
X= df.drop('label',axis=1)

In [50]:
X

,Unnamed: 0,title,text
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello..."
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T..."
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...
...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene..."


In [51]:
y= df['label']

In [52]:
y

0       0
1       0
2       1
3       0
4       1
       ..
6330    1
6331    0
6332    0
6333    1
6334    1
Name: label, Length: 6335, dtype: int32

### Stemming

In [53]:
portstem= PorterStemmer()

In [54]:
def stem(title):
    stemmed_content = re.sub('[^a-zA-Z]',' ',title)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [portstem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [55]:
df['title']= df['title'].apply(stem)

In [56]:
df['title']

0                                      smell hillari fear
1       watch exact moment paul ryan commit polit suic...
2                           kerri go pari gestur sympathi
3          berni support twitter erupt anger dnc tri warn
4                           battl new york primari matter
                              ...                        
6330       state depart say find email clinton specialist
6331                        p pb stand plutocrat pentagon
6332             anti trump protest tool oligarchi inform
6333    ethiopia obama seek progress peac secur east a...
6334                jeb bush suddenli attack trump matter
Name: title, Length: 6335, dtype: object

In [57]:
X = df['title']
y = df['label']

In [58]:
X

0                                      smell hillari fear
1       watch exact moment paul ryan commit polit suic...
2                           kerri go pari gestur sympathi
3          berni support twitter erupt anger dnc tri warn
4                           battl new york primari matter
                              ...                        
6330       state depart say find email clinton specialist
6331                        p pb stand plutocrat pentagon
6332             anti trump protest tool oligarchi inform
6333    ethiopia obama seek progress peac secur east a...
6334                jeb bush suddenli attack trump matter
Name: title, Length: 6335, dtype: object

In [59]:
y

0       0
1       0
2       1
3       0
4       1
       ..
6330    1
6331    0
6332    0
6333    1
6334    1
Name: label, Length: 6335, dtype: int32

### TfidVectorizer

In [60]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [61]:
print(X)

  (0, 5631)	0.7649346276717717
  (0, 2817)	0.31703295849951846
  (0, 2230)	0.5606827254461214
  (1, 6671)	0.27855534550881683
  (1, 6571)	0.21736822482437437
  (1, 6315)	0.13561903129801023
  (1, 5933)	0.3501812302002013
  (1, 5263)	0.29163264316148546
  (1, 4905)	0.29382045650093463
  (1, 4630)	0.24403179029804972
  (1, 4462)	0.26757273402295384
  (1, 3939)	0.3279355637571237
  (1, 2101)	0.44595444447238664
  (1, 1212)	0.3538286538491304
  (2, 6015)	0.5345737509664292
  (2, 4428)	0.37615749278439464
  (2, 3335)	0.41740393299177314
  (2, 2576)	0.3357697230253524
  (2, 2536)	0.5345737509664292
  (3, 6658)	0.3169347988352881
  (3, 6359)	0.3719244073533966
  (3, 6288)	0.31487349957846533
  (3, 5956)	0.2780361491213491
  (3, 2048)	0.44483942568507917
  (3, 1764)	0.35074544989300727
  :	:
  (6330, 1133)	0.2086414599045263
  (6331, 5796)	0.38445288563338503
  (6331, 4611)	0.5530926787707366
  (6331, 4493)	0.45939150101458504
  (6331, 4474)	0.5790025183139886
  (6332, 6315)	0.177445191694796


In [62]:
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size = 0.25,stratify=y ,random_state=2)

### Model Implementation

In [63]:
model= LogisticRegression()

In [64]:
model.fit(X_train, y_train)

LogisticRegression()

### Finding Accuracy Score of the model

In [65]:
Xtrainpred= model.predict(X_train)
trainingdataaccuracy= accuracy_score(Xtrainpred, y_train)

In [66]:
print('Accuracy score of the training data : ', trainingdataaccuracy)

Accuracy score of the training data :  0.9075984003367712


In [67]:
Xtestpred= model.predict(X_test)
testdataaccuracy= accuracy_score(Xtestpred, y_test)

In [68]:
print('Accuracy score of the test data : ', testdataaccuracy)

Accuracy score of the test data :  0.8257575757575758


### Predicting from a new data

In [69]:
Xnew= X_test[4]
pred= model.predict(Xnew)
print(pred)

[1]


### Confusion Matrix

In [70]:
cm = confusion_matrix(y_test, Xtestpred)
cm

array([[683, 108],
       [168, 625]], dtype=int64)

### So the accuracy score of Fake news detection model that we built using Logistic Regression algorithm 
### is 82.58%. 